<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/NorthDakota/hzNorthDakota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dbfread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.1 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 


In [11]:
import pandas as pd
import plotly.express as px
from dbfread import DBF
#import glob
import sqlite3
import sys
import csv
import dbfread
import dataset

In [5]:
!wget https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip

--2022-07-28 12:27:36--  https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip
Resolving gis.dmr.nd.gov (gis.dmr.nd.gov)... 165.234.156.197
Connecting to gis.dmr.nd.gov (gis.dmr.nd.gov)|165.234.156.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293655700 (280M) [application/x-zip-compressed]
Saving to: ‘OGD_Horizontals.zip’

OGD_Horizontals.zip 100%[===================>] 280.05M  84.6MB/s    in 3.5s    

2022-07-28 12:27:40 (79.3 MB/s) - ‘OGD_Horizontals.zip’ saved [293655700/293655700]



In [6]:
!unzip OGD_Horizontals.zip

Archive:  OGD_Horizontals.zip
 extracting: OGD_Horizontals.cpg     
  inflating: OGD_Horizontals.dbf     
  inflating: OGD_Horizontals.prj     
  inflating: OGD_Horizontals.sbn     
  inflating: OGD_Horizontals.sbx     
  inflating: OGD_Horizontals.shp     
  inflating: OGD_Horizontals.shp.xml  
  inflating: OGD_Horizontals.shx     


In [7]:
filename= 'OGD_Horizontals.dbf'



In [8]:
# write to csv file about 2 minutes
table = DBF(filename)
f=open('ndWells.csv', 'w')
writer = csv.writer(f)
writer.writerow(table.field_names)
for record in table:
  writer.writerow(list(record.values()))

In [9]:
%ls

ndWells.csv          OGD_Horizontals.sbn      OGD_Horizontals.shx
OGD_Horizontals.cpg  OGD_Horizontals.sbx      OGD_Horizontals.zip
OGD_Horizontals.dbf  OGD_Horizontals.shp      sample_data/
OGD_Horizontals.prj  OGD_Horizontals.shp.xml


In [12]:
dfWells = pd.read_csv('ndWells.csv')

In [13]:
dfWells.head()

,wl_permit,api_wellno,api_format,bh_long,bh_lat,well_sub,measdpth,bh_inc,bh_azmuth,tvd,bh_coordns,bh_coord_1,bh_coordew,bh_coord_2,apitype
0,1741,33007000260000,33-007-00026-00-00,-103.392973,46.866931,LAT1,9000.0,0.77,4.330000,8999.030273,38.310001,N,35.090000,W,33007000260000_LAT1
1,1741,33007000260000,33-007-00026-00-00,-103.392971,46.866931,LAT1,9015.0,3.30,88.000000,9014.021484,38.425602,N,34.650803,W,33007000260000_LAT1
2,1741,33007000260000,33-007-00026-00-00,-103.392969,46.866931,LAT1,9022.0,5.30,88.000000,9021.000977,38.443920,N,34.126297,W,33007000260000_LAT1
3,1741,33007000260000,33-007-00026-00-00,-103.392964,46.866931,LAT1,9032.0,8.80,88.000000,9030.923828,38.486748,N,32.899883,W,33007000260000_LAT1
4,1741,33007000260000,33-007-00026-00-00,-103.392953,46.866932,LAT1,9047.0,14.00,83.699997,9045.623047,38.726067,N,29.947657,W,33007000260000_LAT1


In [22]:
#dfWells[['api_wellno','bh_lat','bh_long','tvd','well_sub']][dfWells['api_wellno'] == 33007000260000]

In [21]:
plWell = dfWells[['api_wellno','bh_lat','bh_long','tvd','well_sub']][dfWells['api_wellno'] == 33007000260000]
fig = px.scatter_3d(plWell, x='bh_long', y='bh_lat', z='tvd', color='well_sub')
fig.update_traces(marker_size = 3)
fig.update_scenes(yaxis_autorange="reversed")
fig.update_scenes(xaxis_autorange="reversed")
fig.update_scenes(zaxis_range=[16000, 0])
fig.update_layout({
'plot_bgcolor': 'rgba(100, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
# slow, over one hour to process
db = dataset.connect('sqlite:///:memory:')
table = db['wells']
for record in DBF(filename, lowernames=True):
  table.insert(record)
print(db.tables)

In [ ]:
print(db['wells'].columns)

['id', 'wl_permit', 'api_wellno', 'api_format', 'bh_long', 'bh_lat', 'well_sub', 'measdpth', 'bh_inc', 'bh_azmuth', 'tvd', 'bh_coordns', 'bh_coord_1', 'bh_coordew', 'bh_coord_2', 'apitype']


In [ ]:
print(table.find_one(api_wellno = '33007009620000'))

OrderedDict([('id', 5251), ('wl_permit', '10475'), ('api_wellno', '33007009620000'), ('api_format', '33-007-00962-00-00'), ('bh_long', -103.59071073), ('bh_lat', 47.16533453), ('well_sub', 'LAT1'), ('measdpth', 9819.0), ('bh_inc', 10.19999981), ('bh_azmuth', 219.6000061), ('tvd', 9816.98535156), ('bh_coordns', 23.77682495), ('bh_coord_1', 'S'), ('bh_coordew', 2.49027348), ('bh_coord_2', 'E'), ('apitype', '33007009620000_LAT1')])


In [ ]:
plWell = table.find(api_wellno = '33023003280000')


In [ ]:
plWell = table.find(api_wellno = '33007009620000')
fig = px.scatter_3d(plWell, x='bh_long', y='bh_lat', z='tvd', color='well_sub')
fig.update_traces(marker_size = 3)
fig.update_scenes(yaxis_autorange="reversed")
fig.update_scenes(xaxis_autorange="reversed")
fig.update_scenes(zaxis_range=[16000, 0])
fig.update_layout({
'plot_bgcolor': 'rgba(100, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()